In [1]:
import pandas as pd
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

C:\Users\vasub\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('./top_rated_wines.csv')

In [3]:
df.head(5)

,name,region,variety,rating,notes
0,3 Rings Reserve Shiraz 2004,"Barossa Valley, Barossa, South Australia, Aust...",Red Wine,96.0,Vintage Comments : Classic Barossa vintage con...
1,Abreu Vineyards Cappella 2007,"Napa Valley, California",Red Wine,96.0,Cappella is a proprietary blend of two clones ...
2,Abreu Vineyards Cappella 2010,"Napa Valley, California",Red Wine,98.0,Cappella is one of the oldest vineyard sites i...
3,Abreu Vineyards Howell Mountain 2008,"Howell Mountain, Napa Valley, California",Red Wine,96.0,When David purchased this Howell Mountain prop...
4,Abreu Vineyards Howell Mountain 2009,"Howell Mountain, Napa Valley, California",Red Wine,98.0,"As a set of wines, it is hard to surpass the f..."


In [4]:
df.describe(include='all')

,name,region,variety,rating,notes
count,1365,1364,1347,1365.000000,1365
unique,1361,127,5,NaN,1109
top,Chateau de Beaucastel Hommage Jacques Perrin C...,"Napa Valley, California",Red Wine,NaN,
freq,3,152,1154,NaN,13
mean,NaN,NaN,NaN,96.859341,NaN
std,NaN,NaN,NaN,0.995957,NaN
min,NaN,NaN,NaN,96.000000,NaN
25%,NaN,NaN,NaN,96.000000,NaN
50%,NaN,NaN,NaN,97.000000,NaN
75%,NaN,NaN,NaN,98.000000,NaN


In [5]:
print (df.shape)
print (len(df))

(1365, 5)
1365


In [6]:
df['variety'].notna().value_counts()[True]

np.int64(1347)

In [7]:
df = df[df['variety'].notna()] # Remove any NaN values as it blows up serialization
data = df.sample(700).to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)

700

In [8]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [9]:
# Create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [10]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

C:\Users\vasub\AppData\Local\Temp\ipykernel_15196\3075695827.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [11]:
# Vectorize!
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # Data is the variable holding all the wines
    ]
)

In [12]:
# Search time for awesome wines!
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)

C:\Users\vasub\AppData\Local\Temp\ipykernel_15196\3683068012.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(


In [13]:
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Catena Zapata Argentino Vineyard Malbec 2004', 'region': 'Argentina', 'variety': 'Red Wine', 'rating': 98.0, 'notes': '"The single-vineyard 2004 Malbec Argentino Vineyard spent 17 months in new French oak. Remarkably fragrant and complex aromatically, it offers up aromas of wood smoke, creosote, pepper, clove, black cherry, and blackberry. Made in a similar, elegant style, it is the most structured of the three single vineyard wines, needing a minimum of a decade of additional cellaring. It should easily prove to be a 25-40 year wine. It is an exceptional achievement in Malbec. When all is said and done, Catena Zapata is the Argentina winery of reference – the standard of excellence for comparing all others. The brilliant, forward-thinking Nicolas Catena remains in charge, with his daughter, Laura, playing an increasingly large role. The Catena Zapata winery is an essential destination for fans of both architecture and wine in Mendoza. It is hard to believe, given the surge i

In [14]:
# Using it with our own LLM

# Define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [15]:
import os
from openai import AzureOpenAI

In [26]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [27]:
endpoint = os.getenv("ENDPOINT_URL", "https://azure-openai-learning-vasu.openai.azure.com/")  
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4")  
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", os.getenv('OPENAI_API_VERSION_GPT4')) 

In [28]:
# Connect to the large language model

client = AzureOpenAI(  
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version="2024-05-01-preview",
)

In [29]:
chat_prompt = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Suggest me an amazing Malbec wine from Argentina."
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": str(search_results)
            }
        ]
    }
] 

In [30]:
completion = client.chat.completions.create(  
    model=deployment,
    messages=chat_prompt,
    max_tokens=800,  
    temperature=0.7,  
    top_p=0.95,  
    frequency_penalty=0,  
    presence_penalty=0,
    stop=None,  
    stream=False
)

In [ ]:
print(completion.to_json())  

In [ ]:
print(completion.choices[0].message)

In [ ]:
# Chat prompt without the assistant
chat_prompt2 = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Suggest me an amazing Malbec wine from Argentina."
            }
        ]
    }
] 

In [ ]:
completion2 = client.chat.completions.create(  
    model=deployment,
    messages=chat_prompt2,
    max_tokens=800,  
    temperature=0.7,  
    top_p=0.95,  
    frequency_penalty=0,  
    presence_penalty=0,
    stop=None,  
    stream=False
)

In [ ]:
print(completion2.choices[0].message)

# There is some cache so it is picking up context from the previous prompt